# Debt Data Time Series

This script seeks to grab the subset of relevant variables from each year, so that we have a set across all years that can be readily merged with the TEL/ACS data.

In [67]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import requests

## What are the common variables?

The first thing we will do is see if we can get what we need from just the common variables across all sets.  There are a few reasons why variables may not align every year:

1. Reuters doesn't offer the entire set of variables every year;
2. Variable names with long words may have been hyphenated in different ways across years;
3. If variables appeared more than once, the parsing routine appended the variable position to the name to create a unique variable.  If the position varies across years, so will the variable name.

To grab the columns in lightweight fashion, we will just read in the first couple lines for each set.

In [68]:
#Grab list of files
files=glob.glob('../../debt_data/*.csv')

#Create a dictionary to hold columns from each year
col_dict={}

#For each file...
for f in files:
    #...read in the first couple rows...
    tmp_df=pd.read_csv(f,nrows=2)
    #...capture the columns...
    col_dict.update({f[13:-4]:list(tmp_df.columns)})
    #...and dump the partial data set
    del tmp_df
    
#Create a container for the variable sets within each file
var_sets=[]

#For each file...
for f in col_dict.keys():
    #...add the variable set to var_sets
    var_sets.append(set(col_dict[f]))
    
#Capture the intersection of variables across all years
common_vars=sorted(list(set.intersection(*var_sets)))

print 'There are '+str(len(common_vars))+' variables common to all sets.'
print common_vars

There are 254 variables common to all sets.
['# of Mgrs', '$ Amount of Highest Cpn Maturity', '144A FLAG', '501c3', '8-Digit CUSIP25', '8-Digit CUSIP26', 'Accumulator Amt ($ Mil)', 'All Use of Proceeds (Code)', 'All Use of Proceeds (Desc)', 'All Use of Proceeds (Number)', 'Amount at Maturity ($ mils)', 'Amount of Final Maturity ($mils)', 'Amount of Issue ($ mils)', 'Amount of Maturity ($ mils)', 'Ant- ici- pa- tion Type', 'Asset Backed Indicator Flag (Y/N)', 'Auction Rate', 'Aver- age Life', 'Average Take Down', 'Bank Qual', 'Beginning Price/ Yield', 'Beginning Serial Coupon', 'Beginning Serial Maturity', 'Bid', 'Bk Elig', 'Bk En- try', 'Bnk Mgd', 'Bond Buyer ALL UOP', 'Bond Buyer GO Index', 'Bond Buyer Region118', 'Bond Buyer Region119', 'Bond Buyer Rev. Index', 'Bond Buyer UOP142', 'Bond Buyer UOP143', 'Bond Buyer UOP30', 'Bond Counsel Deal(Y/N)', 'CD-ROM Number', 'CUSIP of Insti- tutional Backer', 'Call Date', 'Call Issue', 'Call Price', 'Callable at Par', 'Co-Managers', 'Comb. Gros

## Data Input

Ok, we are looking for aggregations of debt by county.  In particular, we want to capture activity by concepts:

1. Type of Debt (General Obligation or Revenue; latter can be split by )
2. Issuer Type (General purpose gov, school district, special district, or private entity)
3. Purpose of the Issue
4. Volume of Issue

For the latter two, we also want variables that split out GO versus revenue bonds.  For example, we would want to know the volume of GO debt issued by general purpose jurisdictions, or the revenue debt issued in service of transportation infrastructure.  The following table maps concepts to variables.

Concept|Variable|Possible Values
-------|--------|---------------
Debt Type|`Security Type`| GO<br>RV
Issuer Type|`Issuer Type Description`|District<br>City, Town Vlg<br>Local Authority<br>State Authority<br>County/Parish<br>College or Univ<br>State/Province<br>Direct<br>Indian Tribe<br>Co-op Utility
Purpose|`Bond Buyer UOP30`|Development<br>Education<br>Electric Power<br>Environmental Facilities<br>General Purpose<br>Healthcare<br>Housing<br>Public Facilities<br>Transportation<br>Utilities
Volume|`Amount of Maturity (M)`|Continuous
County|`County`|Any county in the US
State|`State`|Any state in the US
Issue Date|`Sale Date`|Continuous (we only need the year)

Fortunately, all of these variables appear in the common set.

In [69]:
#Define required variables
req_vars=['Security Type','Issuer Type Description','Bond Buyer UOP30',\
          'Amount of Maturity ($ mils)','County','State','Sale Date','Issuer']
print 'All the requisite variables are in the common set:',np.array([var in common_vars for var in req_vars]).all()

All the requisite variables are in the common set: True


That makes things easier.  Let's just go ahead and read the data in from all years, keeping only the variables in `req_vars`.

In [70]:
#Create a container for DFs from all years
df_list=[]

#For each file...
for f in files:
    #...throw the subset into df_list
    df_list.append(pd.read_csv(f,usecols=req_vars))
    
#Concatenate all the years together
debt=pd.concat(df_list)

#Convert sale date to datetime
debt['Sale Date']=debt['Sale Date'].apply(lambda x: pd.to_datetime(x))

#Generate a year variable
debt['Year']=debt['Sale Date'].apply(lambda x: x.year)

#Jettison Sale Date
debt.pop('Sale Date')

print debt.info()
debt.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 465391 entries, 0 to 26896
Data columns (total 8 columns):
Bond Buyer UOP30               465388 non-null object
Amount of Maturity ($ mils)    465391 non-null object
County                         461215 non-null object
Issuer                         465357 non-null object
Issuer Type Description        465357 non-null object
State                          465365 non-null object
Security Type                  465391 non-null object
Year                           465391 non-null int64
dtypes: int64(1), object(7)
memory usage: 32.0+ MB
None


,Bond Buyer UOP30,Amount of Maturity ($ mils),County,Issuer,Issuer Type Description,State,Security Type,Year
0,Development,1,Cook/Lake,Buffalo Grove City-Illinois,"City, Town Vlg",IL,RV,1984
1,Education,0.225,Maricopa,Maricopa Co (Fowler) ESD #45,District,AZ,GO,1984
2,General Purpose,1,State Authority,Nevada Dept of Commerce,State Authority,NV,RV,1984
3,General Purpose,0.455,Essex,Essex Co-New York,County/Parish,NY,GO,1984
4,Electric Power,275,Duval,Jacksonville Electric Authority,Local Authority,FL,RV,1984


## Identification of FIPS codes

We need to merge in FIPS codes, which are conveniently held by Census on a public site.

In [71]:
#Define names for fields
fips_names=['state','fips_st','fips_co','county','unknown']

#Capture dtypes of fips code variables (to keep the zeroes)
fips_dtypes={'fips_st':str,
             'fips_co':str}

#Read in fips
fips=pd.read_csv('http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',
                 names=fips_names,dtype=fips_dtypes)

#Remove 'County' and 'Parish' from the county names
fips['county']=fips['county'].str.replace(' County','')
fips['county']=fips['county'].str.replace(' Parish','')

#Create composite name and fips variables
fips['st_cou']=fips.apply(lambda row: (row['state']+'_'+row['county']).lower(),axis=1)
fips['fips']=fips.apply(lambda row: (row['fips_st']+row['fips_co']).lower(),axis=1)

#Capture dict to map composite names to composite fips codes
fips_dict=dict(zip(fips['st_cou'],fips['fips']))

#Fix the counties with 'St' (insert a period)
debt['County']=debt['County'].str.replace('St ','St. ')

#Generate composite name for the debt data
debt['st_cou']=debt.apply(lambda row: (str(row['State'])+'_'+str(row['County'])).lower(),axis=1)

#Map in fips codes
debt['FIPS']=debt['st_cou'].map(fips_dict)

#Write in temporary set (before FIPS improvement)
debt.to_csv('debt_ts_pre_fips.csv')

debt

,Bond Buyer UOP30,Amount of Maturity ($ mils),County,Issuer,Issuer Type Description,State,Security Type,Year,st_cou,FIPS
0,Development,1,Cook/Lake,Buffalo Grove City-Illinois,"City, Town Vlg",IL,RV,1984,il_cook/lake,NaN
1,Education,0.225,Maricopa,Maricopa Co (Fowler) ESD #45,District,AZ,GO,1984,az_maricopa,04013
2,General Purpose,1,State Authority,Nevada Dept of Commerce,State Authority,NV,RV,1984,nv_state authority,NaN
3,General Purpose,0.455,Essex,Essex Co-New York,County/Parish,NY,GO,1984,ny_essex,36031
4,Electric Power,275,Duval,Jacksonville Electric Authority,Local Authority,FL,RV,1984,fl_duval,12031
5,Education,0.635,Maricopa,Maricopa Co SD #92,District,AZ,GO,1984,az_maricopa,04013
6,Development,25,State Authority,New Jersey Economic Dev Auth,State Authority,NJ,RV,1984,nj_state authority,NaN
7,Utilities,0.035,NaN,Farley-Illinois,"City, Town Vlg",IL,GO,1984,il_nan,NaN
8,Utilities,10.51,Fort Bend,First Colony Levee Imp Dt,District,TX,GO,1984,tx_fort bend,48157
9,General Purpose,1.611,Montgomery,Huber Heights City-Ohio,"City, Town Vlg",OH,GO,1984,oh_montgomery,39113


In [72]:
print 'Total number of issues:',len(debt)
print 'Number of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])
print 'Proportion of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])/float(len(debt))

Total number of issues: 465391
Number of issues that still do not have a FIPS code: 101639
Proportion of issues that still do not have a FIPS code: 0.218394855079


### State Issuers

Ok, that got about three quarters of the records.  Let's try to get the rest.  Many issuers come from the `State Authority` or the `State` outright.  Pretty much, if `State` is anywhere in the description, no single county can be affiliated with the issue. So, let's allocate the state FIPS to all of them.

In [73]:
#Capture states
states=sorted(set(debt['State']))[1:]

#For each state...
for st in states:
    #...capture the keys associated with that state...
    st_keys=[item for item in fips_dict.items() if str(st).lower()+'_' in item[0]]
    try:
        #...extract the state portion of the value associated with the first member of the list...
        st_key_part=st_keys[0][1][:2]
        #...and assign the state fips code
        state_in_desc=((debt['Issuer Type Description'].apply(lambda x: 'State' in str(x))) |\
                       (debt['County'].apply(lambda x: 'State' in str(x))))
        st_mask=(debt['State']==st) & (state_in_desc)
        debt.ix[st_mask,'FIPS']=st_key_part+'000'
#         debt.ix[(debt['State']==st) & (debt['County']=='State'),'FIPS']=st_key_part+'000'
    except:
        print 'Problem state >>> ',st

debt[debt['County']=='State Authority']

Problem state >>>  MR
Problem state >>>  TT


,Bond Buyer UOP30,Amount of Maturity ($ mils),County,Issuer,Issuer Type Description,State,Security Type,Year,st_cou,FIPS
2,General Purpose,1,State Authority,Nevada Dept of Commerce,State Authority,NV,RV,1984,nv_state authority,32000
6,Development,25,State Authority,New Jersey Economic Dev Auth,State Authority,NJ,RV,1984,nj_state authority,34000
11,Education,2.46,State Authority,New Jersey Educational Facs Au,State Authority,NJ,RV,1984,nj_state authority,34000
30,General Purpose,10.625,State Authority,Missouri-Illinois Met Dt Bi-St Dev Agy,State Authority,MO,GO,1984,mo_state authority,29000
52,Public Facilities,25,State Authority,Florida Dept of Nat Resources,State Authority,FL,RV,1984,fl_state authority,12000
85,Healthcare,51,State Authority,Arizona Health Facs Authority,State Authority,AZ,RV,1984,az_state authority,04000
97,Transportation,42,State Authority,Washington Metro Area Transit Au,State Authority,DC,GO,1984,dc_state authority,11000
100,Healthcare,4.1,State Authority,NYS Medical Care Facs Fin Agency,State Authority,NY,RV,1984,ny_state authority,36000
148,Housing,3.533,State Authority,Florida Housing Finance Agency,State Authority,FL,RV,1984,fl_state authority,12000
166,Housing,9.155,State Authority,Florida Housing Finance Agency,State Authority,FL,RV,1984,fl_state authority,12000


In [74]:
print 'Total number of issues:',len(debt)
print 'Number of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])
print 'Proportion of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])/float(len(debt))

Total number of issues: 465391
Number of issues that still do not have a FIPS code: 51254
Proportion of issues that still do not have a FIPS code: 0.110131051095


### Using Google to Capture Colleges and Some of the Remaining Misses

Ok, we still have a number of misses.  I considered just taking the first element from hybrid county descriptions, but sometimes the subsequent positions are meaningful.  For example, we wouldn't necessarily get Bronx County if we only captures New York from `New York/Bronx/Kings`.  There are too many to do by hand, so we are going to utilize the Google Maps API to cut down the current gap.  There are a couple lists we have to sort through:

1. Issues with some version of `College or University` as the county description.  We will key on issuer names in this scenario.
2. County descriptions that did not have a direct match in the FIPS data.

The API does not spit out FIPS codes, but it does provide regular county names.  We can construct new keys based upon the state_county combination that hopefully are members of the FIPS mapping dictionary.  First, let's identify our lists.

In [75]:
#Capture list of college issuers
college_pairs=zip(debt[debt['County'].apply(lambda x: 'College' in str(x))]['Issuer'],
                  debt[debt['County'].apply(lambda x: 'College' in str(x))]['State'])
college_issues=list(set(college_pairs))

#Create masks for other county descriptions (excluding authorities)
no_st_auth=((debt['Issuer Type Description'].apply(lambda x: 'State' not in str(x))) |\
            (debt['County'].apply(lambda x: 'State' not in str(x))))
no_loc_auth=(debt['County'].apply(lambda x: 'Local' not in str(x)))
no_college=(debt['County'].apply(lambda x: 'College' not in str(x)))

#Capture list of random issuers
rando_pairs=zip(debt[no_st_auth & no_loc_auth & no_college]['County'],
                debt[no_st_auth & no_loc_auth & no_college]['State'])
rando_issues=list(set(rando_pairs))

len(college_issues)

381

Now we need a function to implement the geocoding...

In [76]:
def county_id(point_of_interest,state=None):
    '''Function returns state_county concatenation of a given point of interest.'''
    #Set base URL
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    #Set parameters for call to API (which are appended to the base)
    params = {'sensor': 'false',
              'address': point_of_interest,
              'key':'Get your own key son!'}
    #Make call to API
    r = requests.get(url, params=params)
    #Capture results
    results = r.json()['results']
#     print results
#     print len(results)
    #If a state is provided...
    if state != None:
        #...for each hit...
        for r in results:
            #...capture the state and county...
            res_st=[comp['short_name'] for comp in r['address_components'] \
                    if comp['types'][0]=='administrative_area_level_1']
            res_co=[comp['short_name'] for comp in r['address_components'] \
                    if comp['types'][0]=='administrative_area_level_2']
            #...if the state matches...
            if res_st[0]==state:
                #...return the county...
                return (res_st[0]+'_'+res_co[0].replace(' County','')).lower()
    else:
        #Capture the state and county
        res_st=[comp['short_name'] for comp in results[0]['address_components'] \
                if comp['types'][0]=='administrative_area_level_1']
        res_co=[comp['short_name'] for comp in results[0]['address_components'] \
                if comp['types'][0]=='administrative_area_level_2']
        #Capture county from the first hit
        return (res_st[0]+'_'+res_co[0].replace(' County','')).lower()

Now, let's roll through these places and capture (hopefully) better keys.  Note that this takes awhile, so we will store our results and read it back the next time.

In [77]:
# print 'Number of College Issues:',len(college_issues)
# print 'Number of Random Issues:',len(rando_issues)

# #Create dicts for colleges and the randos
# college_map={}
# rando_map={}

# #Create containers for misses
# college_miss=[]
# rando_miss=[]

# print '***INITIALIZING COLLEGE LOOP***'
# #For each college...
# for i,college in enumerate(college_issues):
#     if i%50==0:
#         print i,'|',college
#     try:
#         #...capture the new key...
#         college_map.update({college[0]:county_id(college[0]+' '+college[1])})
#     except:
#         college_miss.append(college)

# print '\n\n***INITIALIZING COLLEGE LOOP***'
# #For each random issuer...
# for i,rando in enumerate(rando_issues):
#     if i%50==0:
#         print i,'|',rando
#     try:
#         #...capture the new key...
#         rando_map.update({rando[0]:county_id(rando[0]+' '+rando[1])})
#     except:
#         rando_miss.append(rando)

We can capture the successful API calls in two series.

In [78]:
# #Capture in series
# college_new_keys=Series(college_map)
# rando_new_keys=Series(rando_map)

# #Remove special characters
# ##Tildes
# college_new_keys=college_new_keys.apply(lambda x: x.encode('utf-8').replace('\xc3\xb1','n'))
# rando_new_keys=rando_new_keys.apply(lambda x: x.encode('utf-8').replace('\xc3\xb1','n'))

# #Write to disk
# college_new_keys.to_csv('../data/g_api_college.csv')
# rando_new_keys.to_csv('../data/g_api_rando.csv')

# len(college_new_keys),len(rando_new_keys)

In [79]:
#Read from disk
college_new_keys=pd.read_csv('../data/g_api_college.csv',names=['desc','key'])
rando_new_keys=pd.read_csv('../data/g_api_rando.csv',names=['desc','key'])

#Set indices
college_new_keys.set_index('desc',inplace=True)
rando_new_keys.set_index('desc',inplace=True)

college_new_keys.head()

,key
desc,
Akron University,oh_summit
Alabama State Board of Education,al_clarke
Alabama State University,al_madison
Arizona State University,az_maricopa
Arizona Western College,az_maricopa


We need to map in these new keys, but I would like to preserve the ability to compare the old and new keys.  Consequently, we will create a new `st_cou_g1` variable that will hold the new keys, and a composite variable `st_cou_final` that holds the keys from `st_cou_g1` where they exist, and `st_cou` where they don't.  Note that these should only be assigned where the FIPS code is currently missing.

In [80]:
#Define mask
nofips=(debt['FIPS'].isnull())

#Generate new var
debt['st_cou_g1']=''

#Fill in random new keys
debt.ix[nofips,'st_cou_g1']=debt.ix[nofips]['County'].map(rando_new_keys['key'])

#Fill in college new keys
no_st_cou_g1=(debt['st_cou_g1'].isnull())
debt.ix[nofips & no_st_cou_g1,'st_cou_g1']=debt.ix[nofips & no_st_cou_g1]['Issuer'].map(college_new_keys['key'])

#Create composite variable
debt['st_cou_final']=np.where(debt['st_cou_g1'].notnull(),debt['st_cou_g1'],debt['st_cou'])

#For records without FIPS, use st_cou_final to map in a code
debt.ix[nofips,'FIPS']=debt.ix[nofips]['st_cou_final'].map(fips_dict)

debt[debt['FIPS'].isnull()]

,Bond Buyer UOP30,Amount of Maturity ($ mils),County,Issuer,Issuer Type Description,State,Security Type,Year,st_cou,FIPS,st_cou_g1,st_cou_final
7,Utilities,0.035,NaN,Farley-Illinois,"City, Town Vlg",IL,GO,1984,il_nan,NaN,NaN,il_nan
16,Utilities,0.069,NaN,Epping Sewer Improvement Dt #2,District,ND,RV,1984,nd_nan,NaN,NaN,nd_nan
21,Transportation,0.195,NaN,Norwalk-Illinois,"City, Town Vlg",IL,GO,1984,il_nan,NaN,NaN,il_nan
25,Education,0.205,NaN,Bismark PSD#1,District,NC,GO,1984,nc_nan,NaN,NaN,nc_nan
31,Utilities,0.135,NaN,Sully-Illinois,"City, Town Vlg",IL,GO,1984,il_nan,NaN,NaN,il_nan
34,Transportation,0.177,NaN,Ankeny-Illinois,"City, Town Vlg",IL,RV,1984,il_nan,NaN,NaN,il_nan
45,General Purpose,2,Nome,Nome-Alaska,"City, Town Vlg",AK,GO,1984,ak_nome,NaN,ak_nome,ak_nome
59,General Purpose,0.4,NaN,Itasca-Minnesota,"City, Town Vlg",MN,GO,1984,mn_nan,NaN,NaN,mn_nan
68,General Purpose,0.325,NaN,Seven Devils-North Carolina,"City, Town Vlg",NC,GO,1984,nc_nan,NaN,NaN,nc_nan
71,General Purpose,0.778,NaN,Armstrong Public Service Dt,District,WV,RV,1984,wv_nan,NaN,NaN,wv_nan


In [81]:
print 'Total number of issues:',len(debt)
print 'Number of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])
print 'Proportion of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])/float(len(debt))

Total number of issues: 465391
Number of issues that still do not have a FIPS code: 12763
Proportion of issues that still do not have a FIPS code: 0.0274242518656


### Dealing with composite `County` descriptions

We are down to about 37,000 records with no FIPS codes out of 465,000.  How many of these are because of compound county descriptions (e.g. `New York/Bronx/Kings`)?

In [82]:
debt[debt['FIPS'].isnull()]['st_cou_final'].apply(lambda x: '/' in str(x)).sum()

4440

Looks like we can take out roughly 60% of our misses if one of the locations an actual county name in the FIPS data.  Our approach will be to roll through the compound county descriptions, individually pair them with the associated state, and see if they show up in our keys in `fips_dict`.  While it is not always the case that a county name will show up in the `Issuer` variable, when it does, it appears to correspond with the first jurisdiction mentioned in the `County` variable.  Consequently, our rule will be to take the first match we find.

In [83]:
#Build mask to capture compound counties that do not have FIPS yet
nofips_compound=(debt['FIPS'].isnull()) & (debt['st_cou_final'].apply(lambda x: '/' in str(x)))

#Define function that returns FIPS for county description components
def composite_match(s,delim='/'):
    #If the first jurisdiction is a county in the FIPS set...
    if s.split(delim)[0] in fips_dict.keys():
        #...return the appropriate FIPS code...
        return fips_dict[s.split(delim)[0]]
    #...otherwise...
    else:
        #...capture state...
        s_st=s[:3]
        #...capture other jurisdictions...
        s_jur=s[3:]
        #...and for each remaining jurisdiction...
        for j in s_jur.split(delim):
            #...if one of them shows up in FIPS...
            if s_st+j in fips_dict.keys():
                #...return the appropriate FIPS code
                return fips_dict[s_st+j]
            
#Assign FIPS codes to composite county records
debt.ix[nofips_compound,'FIPS']=debt.ix[nofips_compound]['st_cou_final'].apply(lambda x: composite_match(x))

#Redefine mask for backslash as the delimeter
nofips_compound=(debt['FIPS'].isnull()) & (debt['st_cou_final'].apply(lambda x: '\\' in str(x)))

#Assign FIPS codes to composite county records
debt.ix[nofips_compound,'FIPS']=debt.ix[nofips_compound]['st_cou_final'].apply(lambda x: \
                                                                               composite_match(x,delim='\\'))

debt[debt['FIPS'].isnull()]

,Bond Buyer UOP30,Amount of Maturity ($ mils),County,Issuer,Issuer Type Description,State,Security Type,Year,st_cou,FIPS,st_cou_g1,st_cou_final
7,Utilities,0.035,NaN,Farley-Illinois,"City, Town Vlg",IL,GO,1984,il_nan,NaN,NaN,il_nan
16,Utilities,0.069,NaN,Epping Sewer Improvement Dt #2,District,ND,RV,1984,nd_nan,NaN,NaN,nd_nan
21,Transportation,0.195,NaN,Norwalk-Illinois,"City, Town Vlg",IL,GO,1984,il_nan,NaN,NaN,il_nan
25,Education,0.205,NaN,Bismark PSD#1,District,NC,GO,1984,nc_nan,NaN,NaN,nc_nan
31,Utilities,0.135,NaN,Sully-Illinois,"City, Town Vlg",IL,GO,1984,il_nan,NaN,NaN,il_nan
34,Transportation,0.177,NaN,Ankeny-Illinois,"City, Town Vlg",IL,RV,1984,il_nan,NaN,NaN,il_nan
45,General Purpose,2,Nome,Nome-Alaska,"City, Town Vlg",AK,GO,1984,ak_nome,NaN,ak_nome,ak_nome
59,General Purpose,0.4,NaN,Itasca-Minnesota,"City, Town Vlg",MN,GO,1984,mn_nan,NaN,NaN,mn_nan
68,General Purpose,0.325,NaN,Seven Devils-North Carolina,"City, Town Vlg",NC,GO,1984,nc_nan,NaN,NaN,nc_nan
71,General Purpose,0.778,NaN,Armstrong Public Service Dt,District,WV,RV,1984,wv_nan,NaN,NaN,wv_nan


In [84]:
print 'Total number of issues:',len(debt)
print 'Number of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])
print 'Proportion of issues that still do not have a FIPS code:',len(debt[debt['FIPS'].isnull()])/float(len(debt))

Total number of issues: 465391
Number of issues that still do not have a FIPS code: 8359
Proportion of issues that still do not have a FIPS code: 0.0179612411929


So, we still have over 8000 misses, but that represents under 2% of debt issues.  That's a decent hit rate.  Here's where those misses occur.

In [85]:
print 'Number of counties without FIPS:',len(debt.ix[(debt['FIPS'].isnull())]['County'].value_counts())
debt.ix[(debt['FIPS'].isnull())]['County'].value_counts()

Number of counties without FIPS: 201


College or University          1672
Local Authority                 677
College or Univ                 325
Anchorage                       166
La Salle                        141
Direct Issuer                   136
Baton Rouge                     108
Bossier/Caddo                    75
North Slope                      57
Matanuska-Susitna                36
Fairbanks No Star                32
Hampton Indep City               29
St. Marys                        29
Valdez/Cordova                   27
Barron/Dunn/St. Croix            23
Orleans Parish                   22
Plaquemine                       20
Perham/Dent                      20
District                         19
East Baton Rouge Parish          19
Northwest                        18
Fairbanks                        18
Goge                             16
Sanilac/Lapeer/St. Clair         15
Marshall/St. Joseph              13
Saipan                           12
County/Parish                    12
Ft Pierce                   

In [86]:
#Subset to debt issues with FIPS codes
debt_fips=debt[debt['FIPS'].notnull()]

#Write to disk
debt_fips.to_csv('../data/debt_w_fips.csv')

## Aggregating by County and Year

At this point, we will drop the debt issues we do not have FIPS codes for because they cannot be merged with the institutional data.  Our goal in this section is to generate an output set that captures total volumes of GO and revenue debt issued by county and year.  We will also want the GO and RV debt issued by type of issuer, and the same breakout by purpose.  It is useful to review the table from the beginning of the Notebook here.

Concept|Variable|Possible Values
-------|--------|---------------
Debt Type|`Security Type`| GO<br>RV
Issuer Type|`Issuer Type Description`|District<br>City, Town Vlg<br>Local Authority<br>State Authority<br>County/Parish<br>College or Univ<br>State/Province<br>Direct<br>Indian Tribe<br>Co-op Utility
Purpose|`Bond Buyer UOP30`|Development<br>Education<br>Electric Power<br>Environmental Facilities<br>General Purpose<br>Healthcare<br>Housing<br>Public Facilities<br>Transportation<br>Utilities
Volume|`Amount of Maturity (M)`|Continuous
County|`County`|Any county in the US
State|`State`|Any state in the US
Issue Date|`Sale Date`|Continuous (we only need the year)

We can read this back in here to avoid having to execute the entire Notebook.  We also no longer need FIPS components, so we can drop those.  

*Note:  For the time being we are dropping issues classified as `S` or `T` (as opposed to `GO` and `RV`).  We don't know what they mean currently, and there seven issues affected in the entire data set.  We can revisit this later.*

In [87]:
# #Read in data
# debt_fips=pd.read_csv('../data/debt_w_fips.csv',dtype={'FIPS':str})

# print debt_fips[debt_fips['Security Type'].isin(['S','T'])]

In [88]:
#Read in data
debt_fips=pd.read_csv('../data/debt_w_fips.csv',dtype={'FIPS':str})

#Drop unnecessary variables
for var in ['Unnamed: 0','st_cou','st_cou_g1','st_cou_final']:
    debt_fips.pop(var)
    
#Rename variables
debt_fips.columns=['Purpose','Amount','County','Issuer','Issuer_Type','State','Security_Type','Year','FIPS']

#Subset to exclude S and T Security Types
debt_fips=debt_fips[debt_fips['Security_Type'].isin(['RV','GO'])]

#Convert Amount to float
debt_fips['Amount']=debt_fips['Amount'].apply(lambda x: float(x.replace(',','')))

#Retroactively fix missing Purpose values (see validation effort below)
debt_fips.ix[426160,'Purpose']='Housing'
debt_fips.ix[328463,'Purpose']='General Purpose'
debt_fips.ix[437185,'Purpose']='General Purpose'
    
debt_fips.head()

,Purpose,Amount,County,Issuer,Issuer_Type,State,Security_Type,Year,FIPS
0,Development,1.000,Cook/Lake,Buffalo Grove City-Illinois,"City, Town Vlg",IL,RV,1984,17031
1,Education,0.225,Maricopa,Maricopa Co (Fowler) ESD #45,District,AZ,GO,1984,04013
2,General Purpose,1.000,State Authority,Nevada Dept of Commerce,State Authority,NV,RV,1984,32000
3,General Purpose,0.455,Essex,Essex Co-New York,County/Parish,NY,GO,1984,36031
4,Electric Power,275.000,Duval,Jacksonville Electric Authority,Local Authority,FL,RV,1984,12031


In [89]:
debt_fips.ix[[426160,328463,437185]]

,Purpose,Amount,County,Issuer,Issuer_Type,State,Security_Type,Year,FIPS
426160,Housing,0.45,Kaufman,Kaufman Co MUD #2,District,TX,GO,2013,48257
328463,General Purpose,119.40,Palm Beach,Palm Beach Co School Board,District,FL,RV,2007,12099
437185,General Purpose,9.00,Oswego/Cayuga,Oswego City SD,District,NY,GO,2014,36075


Our approach will be to build this up incrementally.  We will use the appropriate subsets of `debt_fips` to construct three components of the data set (by year and county):

1. Total GO and Revenue debt issue volume;
2. GO and Revenue debt by issuer type; and,
3. GO and Revenue debt by purpose.

These components will then be joined together.

In [90]:
#Capture total debt
tot_debt=debt_fips.groupby(['Year','FIPS','Security_Type']).sum()['Amount'].unstack('Security_Type').fillna(0)

#Define function to capture aggregations by Issuer_Type and Purpose
def debt_by_concept(var):
    #Capture debt by issuer type
    tmp_debt=debt_fips.groupby(['Year','FIPS','Security_Type',var]).sum()['Amount'].sortlevel(2)
    #Unstack types
    tmp_debt=tmp_debt.unstack(['Security_Type',var])
    #Generate new column names
    new_cols=[item[0]+'_'+item[1] for item in tmp_debt.columns.values]
    #Assign new column names
    tmp_debt.columns=new_cols
    #Reorder columns, sort index, and fill in NaN values
    tmp_debt=tmp_debt[sorted(new_cols)].sortlevel(0).fillna(0)
    return tmp_debt

#Capture debt issues for Issuer Type and Purpose tabs
issuer_debt=debt_by_concept('Issuer_Type')
purpose_debt=debt_by_concept('Purpose')

#Join sets together
debt_agg=tot_debt.join([issuer_debt,purpose_debt])

debt_agg

GO       RV  GO_City, Town Vlg  GO_Co-op Utility  \
Year FIPS                                                           
1984 01000    0.000  797.952              0.000                 0   
     01001    0.000    1.625              0.000                 0   
     01003    6.350   23.330              0.000                 0   
     01007    0.000    0.400              0.000                 0   
     01021    1.000    1.425              0.000                 0   
     01025    1.378    0.000              1.378                 0   
     01033    9.674    7.000              9.674                 0   
     01039    0.000    2.225              0.000                 0   
     01043    3.540    0.000              0.000                 0   
     01047    0.000   13.700              0.000                 0   
     01049    3.130   93.065              3.130                 0   
     01055    0.000    8.750              0.000                 0   
     01059    0.000    1.350              0.000                 0   
     01069   12.711    7.600             10.211                 0   
     01071    1.780    0.000              1.780                 0   
     01073   39.992  140.897             10.975                 0   
     01075    0.350    0.000              0.350                 0   
     01077    8.369    9.115              5.869                 0   
     01081    1.983   39.330              1.983                 0   
     01083    0.000    6.050              0.000                 0   
     01089    5.000    0.875              5.000                 0   
     01095    0.000    5.941              0.000                 0   
     01097   23.580  213.685              0.000                 0   
     01101   19.945   95.025             19.945                 0   
     01103    5.000    0.800              5.000                 0   
     01105    0.000   11.000              0.000                 0   
     01109    1.050    0.000              1.050                 0   
     01111    1.500    0.000              0.000                 0   
     01115    0.000    4.000              0.000                 0   
     01117    0.000  102.405              0.000                 0   
...             ...      ...                ...               ...   
2015 55117   19.390    5.500             16.520                 0   
     55119    2.720    0.000              2.720                 0   
     55121   41.440    0.000              4.710                 0   
     55123   20.000    0.000              0.000                 0   
     55125   18.765    0.000              0.000                 0   
     55127   99.170    0.000             15.575                 0   
     55129    3.910    0.000              0.000                 0   
     55131   35.510    0.000             19.145                 0   
     55133  257.040    0.000            118.345                 0   
     55135   11.170    0.000             11.170                 0   
     55137    3.190    0.000              1.690                 0   
     55139   47.575   60.840             38.000                 0   
     55141   32.310    0.000              0.565                 0   
     56000    0.000   77.020              0.000                 0   
     56001    0.000    0.542              0.000                 0   
     56005    0.000    1.252              0.000                 0   
     56009    0.000    0.174              0.000                 0   
     56017    0.000    0.076              0.000                 0   
     56021    0.000    0.516              0.000                 0   
     56025    0.000    0.983              0.000                 0   
     56029    0.000    0.282              0.000                 0   
     56033    0.000   24.972              0.000                 0   
     56039    0.000    0.563              0.000                 0   
     56041    0.000    0.360              0.000                 0   
     56043    0.000   22.040              0.000                 0   
     56045    0

Perhaps we can validate these a bit by ensuring the components add up to the total debt levels for GO and RV respectively.

In [91]:
#Capture subsets
iss_vars={'GO':['GO_City, Town Vlg', 'GO_Co-op Utility','GO_College or Univ', 'GO_County/Parish', 'GO_Direct Issuer',\
                'GO_District', 'GO_Indian Tribe', 'GO_Local Authority','GO_State Authority', 'GO_State/Province'],
          'RV':['RV_City, Town Vlg','RV_Co-op Utility', 'RV_College or Univ', 'RV_County/Parish','RV_Direct Issuer',\
                'RV_District', 'RV_Indian Tribe','RV_Local Authority', 'RV_State Authority', 'RV_State/Province']}
pur_vars={'GO':['GO_Development', 'GO_Education', 'GO_Electric Power','GO_Environmental Facilities', 'GO_General Purpose',\
                'GO_Healthcare', 'GO_Housing', 'GO_Public Facilities','GO_Transportation', 'GO_Utilities'],
          'RV':['RV_Development','RV_Education', 'RV_Electric Power', 'RV_Environmental Facilities','RV_General Purpose',\
                'RV_Healthcare', 'RV_Housing','RV_Public Facilities', 'RV_Transportation', 'RV_Utilities']}

#Create dict to hold comparisons
component_diff={}

#For each variable group...
for i,vg in enumerate([iss_vars,pur_vars]):
    #...and for each Security Type...
    for st in ['GO','RV']:
        #...update the dict with the difference between the component and the reported sums
        component_diff.update({str(i)+'_'+st:debt_agg[st]-debt_agg[vg[st]].sum(axis=1)})

#For each comparison set...
for key in component_diff.keys():
    #...tell me the sum of the misses
    print key,'|',component_diff[key].sum()

1_RV | 205.87
1_GO | 2.0
0_GO | -8.2055473527e-12
0_RV | 1.12403975017e-11


What's going on with the Revenue Bonds by Purpose?  (They were originally off by \$205.87 M. GO bonds by purpose were off by \$2.0 B.)

In [92]:
#Capture index of problem records
idx_rv=component_diff['1_RV'][component_diff['1_RV']>1].index
idx_go=component_diff['1_GO'][component_diff['1_GO']>1].index

#Explore debt_agg at this location
print debt_agg.ix[idx_rv][['RV']+pur_vars['RV']].T
print debt_agg.ix[idx_go][['GO']+pur_vars['GO']].T

Year                            1999     2002
FIPS                           12057    17089
RV                           424.470  239.895
RV_Development                 0.000    4.000
RV_Education                  49.405    0.000
RV_Electric Power              4.175    6.930
RV_Environmental Facilities  138.725    0.000
RV_General Purpose             6.670    0.000
RV_Healthcare                112.060    0.000
RV_Housing                    10.915    0.000
RV_Public Facilities          29.035    0.000
RV_Transportation             56.560   23.470
RV_Utilities                  12.555    3.995
Year                           2002
FIPS                          31153
GO                           76.650
GO_Development                1.335
GO_Education                 29.630
GO_Electric Power             0.000
GO_Environmental Facilities   2.515
GO_General Purpose           24.245
GO_Healthcare                 0.000
GO_Housing                    0.000
GO_Public Facilities          3.400
GO_Tra

Disparity confirmed, and the case of FIPS code 17089 in 2002, the disparity is enormous.  Perhaps the original data can shed some light?

In [93]:
# debt_fips[(debt_fips['Year']==2002) & (debt_fips['FIPS']=='17089')]

Ah, the original data lacked a purpose for a single, very large issue.  I'd wager a similar issue is occurring with FIPS code 12057 in 1999...

In [94]:
# debt_fips[(debt_fips['Year']==1999) & (debt_fips['FIPS']=='12057')]

...and FIPS code 31153 in 2002.

In [95]:
# debt_fips[(debt_fips['Year']==2002) & (debt_fips['FIPS']=='31153')]

Inspection of the original raw data (upstream of `debt_fips`) reveals the following info about our three problematic records:

`debt_fips` Index|Year|FIPS|Issuer|Amount|General Use of Proceeds|Imputed Purpose
-----|----|----|------|------|-----------------------|----------------
328463|1999|12057|Covington Park Comm Dev Dt|4.37|Genl Purpose/ Public Imp|`General Purpose`
426160|2002|17089|Aurora Kane-DuPage Cos-Illinois|201.5|Single Family Housing|`Housing`
437185|2002|31153|Sarpy Co Sanit & Imp Dt #215|2.0|Genl Purpose/ Public Imp|`General Purpose`

*Note: The findings of this little investigation were incorporated, retroactively, into the `debt_fips` set above.  No such issues exist any longer, which is why the prints of the problematic sections of data are commented out.*

In [96]:
len(debt_agg)

61291

## Incorporating Institutional Data

At this point, we are well prepared to join TEL data compiled by Dan and COSTAT/PUMS control variables of interest to our debt data.  These data were provided as SAS files.  The TEL data and COSTAT/PUMS covariate data was joined and converted to CSV in [sas2csv.ipynb](https://github.com/choct155/TELs_debt/blob/master/code/sas2csv.ipynb).  These data share merge keys (year and FIPS codes) with our debt data, which should facilitate their integration.

In [97]:
!ls ../data/

13slsstab1a.xls
2013_GFS_debt.xcf
bonds.csv
costat_mod_vars1940_2010.csv
cty_coverage.csv
current_issue_geocode_list.csv
debt_out.csv
debt_ts_pre_fips.csv
debt_w_fips.csv
fips_st_co_02_07.csv
g_api_college.csv
g_api_rando.csv
geocorr12.csv
shp
state_coverage.csv
tel_data.csv
tel_data_alt.csv


In [98]:
#Read in TEL data
tel=pd.read_csv('../data/tel_data_alt.csv',dtype={'STCOU':str})

#Rename year and FIPS columns
tel=tel.rename(columns={'YEAR':'Year',
                        'STCOU':'FIPS'})
# tel.columns=['Year','FIPS']+list(tel.columns[2:])

#Set index
tel.set_index(['Year','FIPS'],inplace=True)

#Join TEL data to debt data
debt_out=debt_agg.join(tel)

#Write to disk
debt_out.to_csv('../data/debt_out.csv')

print debt_out.ix[1990,'04001'].T.to_string()

GO                             2.261000e+01
RV                             0.000000e+00
GO_City, Town Vlg              0.000000e+00
GO_Co-op Utility               0.000000e+00
GO_College or Univ             0.000000e+00
GO_County/Parish               8.325000e+00
GO_Direct Issuer               0.000000e+00
GO_District                    1.428500e+01
GO_Indian Tribe                0.000000e+00
GO_Local Authority             0.000000e+00
GO_State Authority             0.000000e+00
GO_State/Province              0.000000e+00
RV_City, Town Vlg              0.000000e+00
RV_Co-op Utility               0.000000e+00
RV_College or Univ             0.000000e+00
RV_County/Parish               0.000000e+00
RV_Direct Issuer               0.000000e+00
RV_District                    0.000000e+00
RV_Indian Tribe                0.000000e+00
RV_Local Authority             0.000000e+00
RV_State Authority             0.000000e+00
RV_State/Province              0.000000e+00
GO_Development                 0

It should be noted that the `tel` data is really the integration of two input sets, so `debt_out` is a three-part join.  This has implications for our data universe.  Each set has a different number of counties, and these sets do not necessarily overlap completely.  In the **Data Checks** section of *sas2csv*, we captured the disparity in state coverage in a data set, and wrote it to disk as `cty_coverage.csv` (which we are ultimately concerned with).  Note that in the case of the join in *sas2csv*, we were joining a state level data set (TEL data) to county level data (COSTATS/PUMS).  Therefore, for each state covered in the TEL data, all counties are included by construction.

We can now take the opportunity to check the year-county overlap between the `tel` and `debt_agg` sets.

In [99]:
#Capture union of indices
u_idx=list(set(tel.index.values).union(set(debt_agg.index.values)))

#Generate county coverage dict
cty_cov=DataFrame({'tel':[idx in tel.index for idx in u_idx],
                   'debt_agg':[idx in debt_agg.index for idx in u_idx]},
                    index=pd.MultiIndex.from_tuples(u_idx,names=['Year','FIPS'])).sortlevel(0)

print 'Number of year-counties represented in the TEL set:',cty_cov['tel'].sum()
print 'Number of year-counties represented in the aggregate debt set:',cty_cov['debt_agg'].sum()
print 'Number of complete cases:',len(cty_cov[(cty_cov==True).all(axis=1)])

#Write to disk
cty_cov.to_csv('../data/cty_coverage.csv')

cty_cov.ix[1990:]

Number of year-counties represented in the TEL set: 127405
Number of year-counties represented in the aggregate debt set: 61291
Number of complete cases: 51501


debt_agg    tel
Year FIPS                 
1990 00000    False   True
     01000     True   True
     01001     True   True
     01003     True   True
     01005     True   True
     01007    False   True
     01009    False   True
     01011     True   True
     01013    False   True
     01015     True   True
     01017     True   True
     01019    False   True
     01021     True   True
     01023    False   True
     01025     True   True
     01027    False   True
     01029    False   True
     01031     True   True
     01033     True   True
     01035     True   True
     01037    False   True
     01039    False   True
     01041    False   True
     01043     True   True
     01045    False   True
     01047     True   True
     01049     True   True
     01051    False   True
     01053    False   True
     01055     True   True
...             ...    ...
2015 55117     True  False
     55119     True  False
     55121     True  False
     55123     True  False
     55125     True  False
     55127     True  False
     55129     True  False
     55131     True  False
     55133     True  False
     55135     True  False
     55137     True  False
     55139     True  False
     55141     True  False
     56000     True  False
     56001     True  False
     56005     True  False
     56009     True  False
     56017     True  False
     56021     True  False
     56025     True  False
     56029     True  False
     56033     True  False
     56039     True  False
     56041     True  False
     56043     True  False
     56045     True  False
     60000     True  False
     66000     True  False
     72000     True  False
     78000     True  False

[76796 rows x 2 columns]